<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>From local scikit-learn model to cloud with watson machine learning client</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/raw/master/scikit-learn/hand-written-digits-recognition/images/numbers_banner-04.png" width="600" alt="Icon"> </th>
   </tr>
</table>

This notebook contains steps and code to work with [watson-machine-learning-client](https://pypi.python.org/pypi/watson-machine-learning-client) library available in PyPI repository. This notebook introduces commands for getting data and for basic data exploration, pipeline creation, model training and evaluation, model persistance to Watson Machine Learning repository, model deployment, and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.5, scikit-learn, watson-machine-learning-client package.

You will use a toy dataset available in scikit-learn, **sklearn.datasets.load_digits**, which contains hand-written digits images. Use the toy dataset to recognize hand-written digits.

## Learning goals

The learning goals of this notebook are:

-  Load a sample dataset from scikit-learn.
-  Explore data.
-  Prepare data for training and evaluation.
-  Create Scikit-learn machine learning pipeline.
-  Train and evaluate a model.
-  Persist a model in Watson Machine Learning repository.
-  Deploy model for online scoring using client library.
-  Score sample records using client library.


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Load and explore data](#load)
3.	[Create scikit-learn model](#model)
4.	[Persist model](#persistence)
5.	[Deploy and score in a Cloud](#scoring)
6.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Setup

Before you use the sample code in this notebook, you must perform the following setup task:

-  Create a [Watson Machine Learning Service](https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/) instance (a free plan is offered). 
- Configure local python environment
  + python 3.5
  + scikit-learn 0.17.1
  + xgboost 0.6 (here not used dependency)
  + watson-machine-learning-client

**Tip**: You can install libraries from [PyPI](https://pypi.python.org/pypi).

<a id="load"></a>
## 2. Load and explore data

In this section you will load the data from scikit-learn sample datasets and perform a basic exploration.

In [ ]:
import sklearn
from sklearn import datasets

digits = datasets.load_digits()

Loaded toy dataset consists of 8x8 pixels images of hand-written digits.

Let's display first digit data and label using **data** and **target**.

In [ ]:
print(digits.data[0].reshape((8, 8)))

In [ ]:
digits.target[0]

In next step you will count data examples.

In [ ]:
samples_count = len(digits.images)

print("Number of samples: " + str(samples_count))

<a id="model"></a>
## 3. Create a Scikit-learn model

In this section you will learn how to prepare data, create a Scikit-learn machine learning pipeline, and train a model.

### 3.1: Prepare data

In this subsection you will split your data into: train, test, and score datasets.

In [ ]:
train_data = digits.data[: int(0.7*samples_count)]
train_labels = digits.target[: int(0.7*samples_count)]

test_data = digits.data[int(0.7*samples_count): int(0.9*samples_count)]
test_labels = digits.target[int(0.7*samples_count): int(0.9*samples_count)]

score_data = digits.data[int(0.9*samples_count): ]

print("Number of training records: " + str(len(train_data)))
print("Number of testing records : " + str(len(test_data)))
print("Number of scoring records : " + str(len(score_data)))

As you can see our data has been successfully split into three datasets: 

-  The train data set, which is the largest group, is used for training.
-  The test data set will be used for model evaluation and is used to test the assumptions of the model.
-  The score data set will be used for scoring in Cloud.

### 3.2: Create pipeline and train a model

In this section you will create scikit-learn machine learning pipeline and then train the model.

In the first step you need to import the scikit-learn machine learning packages that will be needed in the subsequent steps.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn import svm, metrics

Standardize features by removing the mean and scaling to unit variance.

In [ ]:
scaler = preprocessing.StandardScaler()

Next, define estimators you want to use for classification. Support Vector Machines (SVM) with radial basis function as kernel is used in the following example.

In [ ]:
clf = svm.SVC(kernel='rbf')

Let's build the pipeline now. A pipeline consists of transformer and an estimator.

In [ ]:
pipeline = Pipeline([('scaler', scaler), ('svc', clf)])

Now, you can train your SVM model by using the previously defined **pipeline** and **train data**.

In [ ]:
model = pipeline.fit(train_data, train_labels)

You can check your **model quality** now. To evaluate the model, use **test data**.

In [ ]:
predicted = model.predict(test_data)

print("Evaluation report: \n\n%s" % metrics.classification_report(test_labels, predicted))

You can tune your model now to achieve better accuracy. For simplicity of this example tuning section is omitted.

<a id="persistence"></a>
## 4. Persist model

In this section you will learn how to store your model in Watson Machine Learning repository by using commont python client.

**Tip**: You can check documentation for watson-machine-learning-client either by visiting [documentation webpage](https://wml-api-pyclient.mybluemix.net) or running below code.

### 4.1: Work with your instance

First, you must import client libraries.

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

Authenticate to Watson Machine Learning service on IBM Cloud.

**Action**: Put authentication information from your instance of Watson Machine Learning service here.</div>

In [ ]:
wml_credentials={
  "url": "https://ibm-watson-ml.mybluemix.net",
  "access_key": "****",
  "username": "****",
  "password": "****",
  "instance_id": "****"
}

**Tip**: Authentication information can be found on **Service Credentials** tab of service instance created on IBM  Cloud. <BR>If you cannot see **instance_id** field in **Serice Credentials** generate new credentials by pressing **New credential (+)** button. 

#### Create API client by running below code.

In [ ]:
client = WatsonMachineLearningAPIClient(wml_credentials)

#### Get instance details.

In [ ]:
import json

instance_details = client.service_instance.get_details()

print(json.dumps(instance_details, indent=2))

### 4.2: Publish model

#### Publish model in Watson Machine Learning repository on Cloud.

Define model name, autor name and email.

In [ ]:
model_props = {"authorName":"IBM", "authorEmail":"ibm@ibm.com"}
model_name = "LOCALLY created Digits prediction model"

In [ ]:
published_model = client.repository.publish(model=model, name=model_name, meta_props=model_props, \
                                            training_data=train_data, training_target=train_labels)

### 4.3: Get model details

In [ ]:
model_details = client.repository.get_details(published_model.uid)

print(json.dumps(model_details, indent=2))

#### Get all models

In [ ]:
models_details = client.repository.get_details()

print(json.dumps(models_details, indent=2))

### 4.3: Load model

In this subsection you will learn how to load back saved model from specified instance of Watson Machine Learning.

In [ ]:
loaded_model = client.repository.load(published_model.uid)

You can make predictions to check that model has been loaded correctly.

In [ ]:
test_predictions = loaded_model.predict(test_data[:10])

In [ ]:
test_predictions

As you can see you are able to make predictions, so model is loaded back correctly. You have already learned how save to and load the model from Watson Machine Learning repository.

### 4.4: Delete model

You can delete published model from Watson Machine Learning repository using below code. The code is commented out at this stage since the model will be needed later for deployment.

In [ ]:
# client.repository.delete(published_model.uid)

<a id="scoring"></a>
## 5. Deploy and score in a Cloud

In this section you will learn how to create online scoring and to score a new data record by using the Watson Machine Learning Client.

### 5.1: Create model deployment

#### Create online deployment for published model

In [ ]:
scoring_endpoint = client.deployments.create(published_model.uid, "Deployment of locally created scikit model")

**Note**: Here we use deployment url saved in published_model object. In next section we show how to retrive deployment url from Watson Mchine Learning instance.

Now you can print an online scoring endpoint. 

In [ ]:
print(scoring_endpoint)

### 5.2: Get deployments

In [ ]:
deployments = client.deployments.get_details()

print(json.dumps(deployments, indent=2))

You can get deployment_url by parsing deployment details for last deployed model.

In [ ]:
deployment_url = deployments.get('resources')[0].get('metadata').get('url')

print(deployment_url)

### 5.3: Score

You can use below method to do test scoring request against deployed model.

**Action**: Prepare scoring payload with records to score.

In [ ]:
scoring_payload = {"values": [list(score_data[0]), list(score_data[1])]}

Use ``client.deployments.score()`` method to run scoring.

In [ ]:
predictions = client.deployments.score(scoring_endpoint, scoring_payload)

In [ ]:
print(json.dumps(predictions, indent=2))

### 5.4: Delete deployment

Use the following method to delete deployment.

In [ ]:
client.deployments.delete(deployment_url)

You can check number of depoyments by running ``client.deployment.get()`` method.

In [ ]:
client.deployments.get_details()

### 5.5: Delete model

In [ ]:
client.repository.delete(published_model.uid)

You can check number of your models by running below ``client.repository.get_details()`` method.

In [ ]:
client.repository.get_details()

<a id="summary"></a>
## 6. Summary and next steps     

 You successfully completed this notebook! You learned how to use scikit-learn machine learning as well as Watson Machine Learning for model creation and deployment. Check out our _[Online Documentation](https://console.ng.bluemix.net/docs/services/PredictiveModeling/index.html?pos=2)_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Wojciech Sobala**, Data Scientist in IBM developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Copyright © 2017 IBM. This notebook and its source code are released under the terms of the MIT License.